In [ ]:
# 1. Arbeitsverzeichnis erstellen
now = strftime('%d-%m-%Y-%H_%M_%S', localtime())
rootDir = './'
experimentDir = f'{rootDir}Versuch_{now}/'

mask = None

if createDir(experimentDir):
    printt(f'Die Ergebnisse werden im Verzeichnis {experimentDir} gespeichert.')
else:
    raise RuntimeError('Fehler: Verzeichnis konnte nicht erstellt werden!')

# 2. Kamera starten
with Vimba() as vimba:
    cameraIds = vimba.camera_ids()
    
    if (len(cameraIds) == 0):
        raise RuntimeError(f'Es wurde keine Kamera gefunden. Bitte stelle sicher, dass die Kamera angeschlossen ist.')
   
    printt(f'Folgenden Kameras wurden gefunden: {cameraIds}')
    cameraId = cameraIds[0]
    camera = vimba.camera(cameraId)
        
    try:
        camera.open()
    except VimbaException as e:
        raise RuntimeError(f'Die Kamera {cameraId} konnte nicht geöffnet werden, vermutlich wird sie von einem anderen Programm verwendet.')
    
    printt('Verwende folgende Kamera-Konfiguration:')
    for config in configList:
        printt(f'  {config[0]}: {config[1]}')
        feature = camera.feature(config[0])
        feature.value = config[1]
        
    _frameDiffMin = -1
    _frameDiffList = []
    _frameId = 0
    _lastFrameBinary = None
    _dropTime = 0
    
    _trackbarValue = 0
    
    _windowNameLiveView = 'Echtzeit-Ansicht'
    _frame = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH), np.uint8)
    _liveViewRunning = False
    
    CALIBRATION_FRAME_COUNT = 50
    TRIGGER_FACTOR = 10
    DROP_FALL_DURATION = 20 # Sekunden
    
    preparationRunning = True
    createMask = False
    captureRunning = False
    
    printt('Starte Kamera...')
    camera.arm('Continuous', processFrame)
    camera.start_frame_acquisition()
    
    while preparationRunning:
        printt('Vorbereitung läuft')
        showLiveView('Vorbereitung: Echtzeit-Ansicht')
        
        if askYesNoQuestion('Aufnahme starten (J/N)?'):
            preparationRunning = False
            createMask = True
        elif askYesNoQuestion('Echtzeit-Ansicht wieder öffnen (J/N)?'):
            pass
        else:
            preparationRunning = False
            
    if createMask:
        printt('Maske erstellen...')
        showMessageBox('Nächster Schritt', 'Bitte lege den für die Aufnahme interessanten Bereich fest')
        frameMask = _frame
        points = askUserForMask(frameMask)
        pointCount = len(points)

        if pointCount < 4:
            showMessageBox('Fehler', 'Zu wenige Masken-Eckpunkte')
        elif not (points[0] == points[pointCount - 1]):
            showMessageBox('Fehler', 'Der Pfad muss geschlossen sein')
        else:
            mask = np.zeros(sizeOf(frameMask, heightFirst = True), np.uint8)
            cv.fillPoly(mask, np.array([points]), (255))
        
        captureRunning = mask is not None or askYesNoQuestion('Ohne Maske fortfahren (J/N)?')

    while captureRunning:
        printt('Aufnahme läuft')
        showLiveView('Aufnahme: Echtzeit-Ansicht')
        
        if askYesNoQuestion('Aufnahme beenden (J/N)?'):
            captureRunning = False
    
    printt('Beende Kamera...')
    camera.stop_frame_acquisition()
    camera.disarm()
    camera.close()
    
    printt('Programm beendet')

## Allgemeine Einstellungen

In [ ]:
import cv2 as cv
import numpy as np
from pymba import Vimba, Frame, VimbaException
from datetime import datetime
from time import localtime, strftime, gmtime, sleep
import tkinter as tk
from tkinter import messagebox
import math
import time
import os

def printt(message):
    now = datetime.now().time().strftime('%H:%M:%S.%f')[:-3]
    print(f'{now}: {message}')
    
def inputt(message):
    now = datetime.now().time().strftime('%H:%M:%S.%f')[:-3]
    return input(f'{now}: {message}')

In [ ]:
import cv2 as cv, numpy as np, math

## Kamera-Konfiguration

In [ ]:
IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 960

configList = [
    ('PixelFormat',  'Mono8'),
    ('OffsetX',      0),
    ('OffsetY',      0),
    ('Height',       IMAGE_HEIGHT),
    ('Width',        IMAGE_WIDTH),
    ('ExposureAuto', 'Off'),
    ('ExposureTime', 2000),
    ('AcquisitionFrameRate', 20)
]

## Frame-Verarbeitung

In [ ]:
def processFrame(frameData):
    global _frame, _frameId, _lastFrameBinary, _dropTime, _frameDiffMin, _frameDiffList
    _frame = frameData.buffer_data_numpy()
    
    if captureRunning:
        frameBlur = cv.GaussianBlur(_frame, (5, 5), 0)
        histogram = calcHistogram(frameBlur, mask)
        threshold, _, _ = calcThreshold(histogram)
        _, frameBinary = cv.threshold(_frame, threshold, 255, cv.THRESH_BINARY)
        
        if not isBackgroundBlack(frameBinary):
            frameBinary = cv.bitwise_not(frameBinary)
        
        if mask is not None:
            frameBinary = cv.bitwise_and(frameBinary, mask)
            
        if isDropFalling(frameBinary):
            _dropTime = round(time.time())
            
        if round(time.time()) < _dropTime + DROP_FALL_DURATION:
            saveFrame = True
        else:
            if _lastFrameBinary is None:
                saveFrame = True
            else:
                frameDiff = cv.bitwise_xor(frameBinary, _lastFrameBinary)
                diffRelative = cv.countNonZero(frameDiff) / (IMAGE_WIDTH * IMAGE_HEIGHT)
                
                if _frameDiffMin == -1:
                    saveFrame = True
                    
                    if len(_frameDiffList) < CALIBRATION_FRAME_COUNT:
                        _frameDiffList.append(diffRelative)
                    else:
                        _frameDiffMin = sum(_frameDiffList) / len(_frameDiffList)
                        printt(f'Kalibirierung abgeschlossen. Mittleres Rauschen: {_frameDiffMin}, maximales Rauschen: {max(_frameDiffList)}')
                else:
                    saveFrame = diffRelative >= _frameDiffMin * TRIGGER_FACTOR
        
        if saveFrame:
            _lastFrameBinary = frameBinary
            now = datetime.now().time().strftime('%H_%M_%S_%f')[:-3]
            fileName = f'{experimentDir}{_frameId}-{now}.bmp'
            cv.imwrite(fileName, _frame)
            
    if _liveViewRunning:
        if captureRunning:
            cv.rectangle(_frame, (_trackbarValue, 0), (_trackbarValue + 1, 0 + IMAGE_HEIGHT), (255))
        else:
            cv.rectangle(_frame, (0, IMAGE_HEIGHT // 2 - 1), (IMAGE_WIDTH, IMAGE_HEIGHT // 2 + 1), (255))
        cv.imshow(_windowNameLiveView, _frame)
        
    _frameId += 1

## Echtzeit-Ansicht

In [ ]:
def showLiveView(windowName = 'Echtzeit-Ansicht'):
    global _windowNameLiveView, _liveViewRunning
    
    cv.namedWindow(_windowNameLiveView)
    printt('Drücke eine beliebige Taste, um die Echtzeit-Ansicht zu beenden')
    
    _liveViewRunning = True
    showImage(_frame, _windowNameLiveView)
    _liveViewRunning = False

## Allgemeine Helfer

In [ ]:
# Berechnet die Distanz zwischen zwei Punkten
def calcDistance(point1, point2):
    dX = point1[0] - point2[0]
    dY = point1[1] - point2[1]
    return math.sqrt(dX**2 + dY**2)

# Gibt die Größe des Bildes image zurück
def sizeOf(image, heightFirst = False):
    if heightFirst:
        return (image.shape[0], image.shape[1])
    else:
        return (image.shape[1], image.shape[0])
    
# Zeigt ein Bild in einem separaten Fenster an
def showImage(image, title='Bild'):
    cv.imshow(title, image)
    k = cv.waitKey(0)
    cv.destroyAllWindows()
        
# Zeigt ein einfaches Hinweise-Fenster an
def showMessageBox(title, message, warning = False):
    rootWindow = tk.Tk()
    rootWindow.withdraw()
    if warning:
        tk.messagebox.showwarning(title, message)
    else:
        tk.messagebox.showinfo(title, message)
    rootWindow.destroy()

# Zeigt eine Frage in der Eingabezeile an
def askYesNoQuestion(question):
    answer = inputt(question)
    return answer == 'j' or answer == 'J'

# Ein Verzeichnis erstellen, falls es noch nicht existiert
def createDir (directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
        return True
    except OSError as e:
        print (f'Fehler: Konnte Verzeichnis {directory} nicht erstellen. Grund: {str(e)}')

## Maske erstellen

In [ ]:
POINT_RADIUS = 8
LINE_THICKNESS = 4
WINDOW_NAME_MASK = 'Maske erstellen'

_points = [] # Die Eckpunkte der Maske
_image = None
_imagePreview = None # Bild, das während der Erstellung der Maske angezeigt wird

def askUserForMask(image):
    global _image, _imagePreview, _points
    
    _points = []
    _image = image
    _imagePreview = image.copy()
    
    cv.namedWindow(WINDOW_NAME_MASK)
    cv.setMouseCallback(WINDOW_NAME_MASK, handleMouseEvent)
    
    trackbarName = 'Position'
    cv.createTrackbar(trackbarName, WINDOW_NAME_MASK , 0, IMAGE_WIDTH, handleTrackbarEvent)
    
    showImage(_imagePreview, WINDOW_NAME_MASK)    
    return _points

def handleMouseEvent(event, x, y, flags, param):
    global _imagePreview, _points
    
    if event == cv.EVENT_LBUTTONUP:
        pointCount = len(_points)
        if pointCount > 0:
            distance = calcDistance(_points[0], (x, y))
            if distance > POINT_RADIUS:
                _points.append((x, y))
                cv.line(_imagePreview, _points[pointCount - 1], _points[pointCount], (255), LINE_THICKNESS, cv.LINE_AA)
                cv.circle(_imagePreview, (x, y), POINT_RADIUS, 255, -1, cv.LINE_AA)
                cv.imshow(WINDOW_NAME_MASK, _imagePreview)
            else:
                _points.append(_points[0])
                cv.line(_imagePreview, _points[pointCount - 1], _points[0], (255), LINE_THICKNESS, cv.LINE_AA)
                cv.imshow(WINDOW_NAME_MASK, _imagePreview)
                if pointCount > 2:
                    showMessageBox('Gut gemacht!', 'Maske erfolgreich erstellt')
                    cv.destroyAllWindows()
                else:
                    showMessageBox('Kann das sein?', 'Bitte setze mehr Eckpunkte', warning = True)
        else:
            _points.append((x, y))
            cv.circle(_imagePreview, (x, y), POINT_RADIUS, 255, -1, cv.LINE_AA)
            cv.imshow(WINDOW_NAME_MASK, _imagePreview)
            
    elif event == cv.EVENT_RBUTTONUP:
        if len(_points) > 0:
            _points.pop()
            _imagePreview = _image.copy()
            drawRect(_trackbarValue, _imagePreview)
            drawPath(_points, _imagePreview)
            cv.imshow(WINDOW_NAME_MASK, _imagePreview)
            
def handleTrackbarEvent(value):
    global _trackbarValue, _imagePreview
    _trackbarValue = value
    _imagePreview = _image.copy()
    drawRect(_trackbarValue, _imagePreview)
    drawPath(_points, _imagePreview)
    cv.imshow(WINDOW_NAME_MASK, _imagePreview)
            
def drawPath(points, imagePreview):
    pointsCount = len(points)
    
    for i in range(pointsCount):
        if i > 0:
            cv.line(imagePreview, points[i - 1], points[i], (255), LINE_THICKNESS, cv.LINE_AA)
        cv.circle(imagePreview, points[i], POINT_RADIUS, 255, -1, cv.LINE_AA)
        
def drawRect(x, imagePreview):
    cv.rectangle(imagePreview, (x, 0), (x + 1, 0 + IMAGE_HEIGHT), (255))

## Binarisierung

In [ ]:
def calcHistogram(image, mask = None):
    return cv.calcHist([image], [0], mask, [256], [0, 256])

def calcThreshold(histogram):
    histogramNormalized = histogram.ravel() / histogram.sum()
    C = np.sum(np.multiply(np.arange(256), histogramNormalized))

    p0, m0 = 0, 0
    variance = np.empty(256)
    varianceMax, threshold = -1, -1

    for i in range(256):
        p0 += histogramNormalized[i]
        m0 += histogramNormalized[i] * i

        q0 = m0 / p0 if p0 > 1.e-6 else 0
        q1 = (C - m0) / (1 - p0)
    
        variance[i] = p0 * (1 - p0) * (q0 - q1)**2
    
        if variance[i] > varianceMax:
            varianceMax = variance[i]
            threshold = i

    return threshold, varianceMax, variance

def isBackgroundBlack(image):
    size = sizeOf(image)
    sizeRect = 10
    x, y = int(size[0] / 2), int(size[1] / 5)
    rect = image[y:y+sizeRect, x:x+sizeRect]
    brightness = np.sum(rect) / sizeRect**2

    return brightness <= 127

def isDropFalling(image):
    size = sizeOf(image)
    x, y = _trackbarValue, 150 # Position für x bestimmen!
    rect = image[y:IMAGE_HEIGHT - 300, x:x + 1]
    count = np.sum(rect) / 255

    return count >= 4